In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from functools import reduce

import sys
sys.path.append("../")

# local imports


In [4]:
train_df = pd.read_csv("../data/cs-training.csv").drop("Unnamed: 0", axis = 1)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7   NumberOfTimes90DaysLate               150000 non-null  int64  
 8   NumberRealEstateLoansOrLines          150000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 10  NumberOfDependents                    146076 non-null  float64
dtype

In [ ]:
delinquency_columns = [
 'NumberOfTime30-59DaysPastDueNotWorse',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfTimes90DaysLate']


train_df.loc[:,"SumTotalTimesPastDue"] = train_df[delinquency_columns].sum(axis = 1)
train_df.loc[:,"MeanTotalTimesPastDue"] = train_df[delinquency_columns].mean(axis = 1)
train_df.loc[:,"StdTotalTimesPastDue"] = train_df[delinquency_columns].std(axis = 1)
train_df.loc[:,"MaxTotalTimesPastDue"] = train_df[delinquency_columns].max(axis = 1)
train_df.loc[:,"MinTotalTimesPastDue"] = train_df[delinquency_columns].min(axis = 1)

train_df.loc[:,"PercentageNumberTimes30to59PastDue"] = (train_df["NumberOfTime30-59DaysPastDueNotWorse"]/
                                                             train_df["SumTotalTimesPastDue"]
                                         )
train_df.loc[:,"PercentageNumberTimes60to89PastDue"] = (train_df["NumberOfTime60-89DaysPastDueNotWorse"]/
                                                             train_df["SumTotalTimesPastDue"]
                                         )
train_df.loc[:,"PercentageNumberTimes90plusPastDue"] = (train_df["NumberOfTimes90DaysLate"]/
                                                             train_df["SumTotalTimesPastDue"]
                                         )

train_df.loc[:,"LogDifference30to5960to89"] = np.log(train_df["NumberOfTime30-59DaysPastDueNotWorse"]) - np.log(train_df["NumberOfTime60-89DaysPastDueNotWorse"])
train_df.loc[:,"LogDifference60to8990plus"] = np.log(train_df["NumberOfTime60-89DaysPastDueNotWorse"]) - np.log(train_df["NumberOfTimes90DaysLate"])


train_df['IncomePerCreditLineLoan'] = train_df['MonthlyIncome'] / train_df['NumberOfOpenCreditLinesAndLoans']
train_df['IncomePerRealEstateLoan'] = train_df['MonthlyIncome'] / train_df['NumberRealEstateLoansOrLines']
train_df['IncomePerDependent'] = train_df['MonthlyIncome'] / train_df['NumberOfDependents']
train_df['IncomePerAge'] = train_df['MonthlyIncome'] / train_df['age']
train_df['DependentsPerAge'] = train_df['NumberOfDependents'] / train_df['age']
train_df['RealEstateLoansPerCreditLineLoan'] = train_df['NumberRealEstateLoansOrLines'] / train_df['NumberOfOpenCreditLinesAndLoans']
train_df['UtilizationTimesCreditLineLoan'] = train_df['RevolvingUtilizationOfUnsecuredLines'] * train_df['NumberOfOpenCreditLinesAndLoans']
train_df['IncomeTimesUtilization'] = train_df['MonthlyIncome'] * train_df['RevolvingUtilizationOfUnsecuredLines']
train_df['UtilizationOverDebtRatio'] = train_df['RevolvingUtilizationOfUnsecuredLines'] / train_df['DebtRatio']
train_df['IncomeOverDebtRatio'] = train_df['MonthlyIncome'] / train_df['DebtRatio']
train_df['SumTotalTimesPastDuePerCreditLineLoan'] =  train_df['NumberOfOpenCreditLinesAndLoans']/train_df['SumTotalTimesPastDue']

In [41]:
%load_ext autoreload
%autoreload 2

pd.set_option("display.max_columns",None)

from preprocess.features.make_features import main

train_df = pd.read_csv("../data/cs-training.csv").drop("Unnamed: 0",axis = 1)
test_df = pd.read_csv("../data/cs-test.csv").drop("Unnamed: 0",axis = 1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
 _, bins = pd.qcut(train_df[socio], q=10, labels=False, retbins=True, duplicates = "drop")
 _mapa = dict(train_df.groupby(bins)[credit].agg(agg_fn))
 train_df.loc[:,f"CategoryEncoded_{socio}_{agg_fn}_{credit}"]= pd.cut(train_df[socio], bins=bins, labels=False).map(_mapa)
 test_df.loc[:,f"CategoryEncoded_{socio}_{agg_fn}_{credit}"] = pd.cut(test_df[socio], bins=bins, labels=False).map(_mapa)

In [51]:
cuts, bins = pd.qcut(train_df["age"], q=10, labels=False,retbins=True, duplicates = "drop")

In [52]:
_mapa =dict(train_df.groupby(cuts)["NumberOfOpenCreditLinesAndLoans"].agg("mean"))

In [54]:
pd.cut(train_df["age"], bins=bins, labels=False).map(_mapa)

0         3.0
1         2.0
2         1.0
3         0.0
4         4.0
         ... 
149995    9.0
149996    2.0
149997    6.0
149998    0.0
149999    7.0
Name: age, Length: 150000, dtype: float64

In [36]:
df.min().min()

-2.6390573296152584